In [1]:
from Data_manager.UserUtils import *
from Data_manager.split_functions.split_train_validation_random_holdout import \
    split_train_in_two_percentage_global_sample
URM_all = getURM_all()
URM_train_validation, URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.80)
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train_validation, train_percentage = 0.80)

In [2]:
from Recommenders.KNN.UserKNNCFRecommender import UserKNNCFRecommender
model = UserKNNCFRecommender

from Evaluation.Evaluator import EvaluatorHoldout
evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10], ignore_users=[])

EvaluatorHoldout: Ignoring 2595 (20.5%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users


In [3]:
import optuna as op
def objective(trial):
    topK = trial.suggest_int("topK", 1e2, 1e4)
    shrink = trial.suggest_int("shrink", 0, 100)
    similarity = trial.suggest_categorical("similarity", ["cosine", "jaccard", "asymmetric", "dice", "tversky"])
    normalize = trial.suggest_categorical("normalize", [True, False])
    feature_weighting = trial.suggest_categorical("feature_weighting", ["none", "BM25", "TF-IDF"])
    recommender = model(URM_train)
    if feature_weighting == "tversky": 
        tversky_alpha = trial.suggest_float("tversky_alpha", 0, 2)
        tversky_beta = trial.suggest_float("tversky_beta", 0, 2)
        normalize = True
        tversky_params = [tversky_alpha, tversky_beta, normalize]
        recommender.fit(**trial.params, **tversky_params)

    else :
        recommender.fit(**trial.params)
    
    result, _ = evaluator_validation.evaluateRecommender(recommender)
    MAP_result = result["MAP"].item()
       
    return MAP_result

In [4]:
best_params = {
    'topK': 529,
    'shrink': 45,
    'similarity': 'asymmetric',
    'normalize': True,
    'feature_weighting': 'TF-IDF',
    'tversky_alpha': 0.0,
    'tversky_beta': 0.0,
}
study = op.create_study(direction="maximize")
study.enqueue_trial(best_params)
study.optimize(objective, n_trials=100)

[I 2023-11-27 14:28:25,229] A new study created in memory with name: no-name-1af6196b-9225-4bd2-b7f3-6de87c86a567


UserKNNCFRecommender: URM Detected 468 ( 3.7%) users with no interactions.
UserKNNCFRecommender: URM Detected 305 ( 1.4%) items with no interactions.
Similarity column 12638 (100.0%), 9455.52 column/sec. Elapsed time 1.34 sec
EvaluatorHoldout: Processed 10043 (100.0%) in 6.84 sec. Users per second: 1469


[I 2023-11-27 14:28:33,549] Trial 0 finished with value: 0.023874364044134025 and parameters: {'topK': 529, 'shrink': 45, 'similarity': 'asymmetric', 'normalize': True, 'feature_weighting': 'TF-IDF'}. Best is trial 0 with value: 0.023874364044134025.


UserKNNCFRecommender: URM Detected 468 ( 3.7%) users with no interactions.
UserKNNCFRecommender: URM Detected 305 ( 1.4%) items with no interactions.
Similarity column 12638 (100.0%), 9764.93 column/sec. Elapsed time 1.29 sec
EvaluatorHoldout: Processed 10043 (100.0%) in 7.01 sec. Users per second: 1433


[I 2023-11-27 14:28:42,099] Trial 1 finished with value: 0.017758270389705223 and parameters: {'topK': 920, 'shrink': 21, 'similarity': 'jaccard', 'normalize': True, 'feature_weighting': 'BM25'}. Best is trial 0 with value: 0.023874364044134025.


...

[I 2023-11-27 14:44:10,194] Trial 97 finished with value: 0.01922687206915026 and parameters: {'topK': 1817, 'shrink': 46, 'similarity': 'asymmetric', 'normalize': True, 'feature_weighting': 'TF-IDF'}. Best is trial 92 with value: 0.02391316940331183.


UserKNNCFRecommender: URM Detected 468 ( 3.7%) users with no interactions.
UserKNNCFRecommender: URM Detected 305 ( 1.4%) items with no interactions.
Similarity column 12638 (100.0%), 7725.15 column/sec. Elapsed time 1.64 sec
EvaluatorHoldout: Processed 10043 (100.0%) in 7.42 sec. Users per second: 1354


[I 2023-11-27 14:44:19,495] Trial 98 finished with value: 0.021216732336666683 and parameters: {'topK': 857, 'shrink': 40, 'similarity': 'dice', 'normalize': True, 'feature_weighting': 'TF-IDF'}. Best is trial 92 with value: 0.02391316940331183.


UserKNNCFRecommender: URM Detected 468 ( 3.7%) users with no interactions.
UserKNNCFRecommender: URM Detected 305 ( 1.4%) items with no interactions.
Similarity column 12638 (100.0%), 9136.06 column/sec. Elapsed time 1.38 sec
EvaluatorHoldout: Processed 10043 (100.0%) in 6.60 sec. Users per second: 1521


[I 2023-11-27 14:44:27,652] Trial 99 finished with value: 0.023903939251693918 and parameters: {'topK': 457, 'shrink': 63, 'similarity': 'cosine', 'normalize': True, 'feature_weighting': 'TF-IDF'}. Best is trial 92 with value: 0.02391316940331183.


In [5]:
study.best_params

{'topK': 529,
 'shrink': 45,
 'similarity': 'asymmetric',
 'normalize': True,
 'feature_weighting': 'TF-IDF'}

In [7]:
final = model(URM_train_validation)
final.fit(**study.best_params)

SLIMElasticNetRecommender: URM Detected 232 ( 1.8%) users with no interactions.
SLIMElasticNetRecommender: URM Detected 103 ( 0.5%) items with no interactions.
SLIMElasticNetRecommender: Processed 17389 (78.3%) in 5.00 min. Items per second: 57.96
SLIMElasticNetRecommender: Processed 22222 (100.0%) in 6.45 min. Items per second: 57.42


In [8]:
from Evaluation.Evaluator import EvaluatorHoldout
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10], ignore_users=[])
evaluator_test.evaluateRecommender(final)

EvaluatorHoldout: Ignoring 2176 (17.2%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Processed 10462 (100.0%) in 6.56 sec. Users per second: 1596


(       PRECISION PRECISION_RECALL_MIN_DEN    RECALL       MAP MAP_MIN_DEN  \
 cutoff                                                                      
 10      0.096683                 0.167108  0.140668  0.049947    0.086206   
 
             MRR      NDCG      F1  HIT_RATE ARHR_ALL_HITS  ... COVERAGE_USER  \
 cutoff                                                     ...                 
 10      0.27832  0.154074  0.1146  0.527337      0.368245  ...      0.827821   
 
        COVERAGE_USER_HIT USERS_IN_GT DIVERSITY_GINI SHANNON_ENTROPY  \
 cutoff                                                                
 10              0.436541    0.827821       0.054661        10.16803   
 
        RATIO_DIVERSITY_HERFINDAHL RATIO_DIVERSITY_GINI RATIO_SHANNON_ENTROPY  \
 cutoff                                                                         
 10                       0.997643             0.156901              0.783344   
 
        RATIO_AVERAGE_POPULARITY RATIO_NOVELTY  
 cutoff